# Facebook AI Similarity Search

Comparing two vectors is easy. Cosine similarity is a simple and effective technique. But what if you have thousands or even millions of vectors? How do you find the most similar vectors to a given query vector? Time required to compare (via cosine similarity) with all vectors scales linearly, so comparing to a hundred vectors is not a problem but even our simple searches through plays in the previous workbook took ~200ms to run. On a much larger corpus this would take seconds or tens of seconds for a search. This is the problem that similarity search solves. Running cosine similarity over such a large set of vectors is not feasible, so we need to use other approaches. This is what [Facebook AI Similarity Search (FAISS)](https://ai.meta.com/tools/faiss/) does. It uses indexing to speed up the search process.

Available indexes are documented at https://github.com/facebookresearch/faiss/wiki/Faiss-indexes

## Motivation

In [ ]:
from sentence_transformers import SentenceTransformer

# Available models: https://www.sbert.net/docs/pretrained_models.html
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
# model = SentenceTransformer('bert-base-nli-mean-tokens')
# model = SentenceTransformer('multi-qa-distilbert-cos-v1')
# model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

In [ ]:
# Corpus of documents to search
import os

directory = ".\plays"
files = {}

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as file:
            file_name = os.path.splitext(filename)[0]
            file_contents = file.read()
            files[file_name] = file_contents

For FAISS exploration, let's use a small chunk size so that we get many vectors.

In [ ]:
import semchunk
import tiktoken

class Chunk:
    def __init__(self, text, doc_name, embedding):
        self.text = text
        self.doc_name = doc_name
        self.embedding = embedding

# Chunk the files into smaller portions
corpus_chunks = {}
doc_names = list(files.keys())
encoder = tiktoken.encoding_for_model('gpt-4')

print("Chunking plays...")
for play, text in files.items():
    corpus_chunks[play] = [Chunk(t, play, None) for t in semchunk.chunk(text, chunk_size = 64, token_counter=lambda text: len(encoder.encode(text)))]

print("Embedding chunks...")
for play in doc_names:
    for chunk in corpus_chunks[play]:
        chunk.embedding = model.encode(chunk.text)

print("Complete")

for play in doc_names:
    print(f'{play}: {len(corpus_chunks[play])} chunks')

In [ ]:
# Combine all play's chunks into a single list
chunks = [chunk for play in doc_names for chunk in corpus_chunks[play]]
print(sum([len(corpus_chunks[play]) for play in doc_names]))
print(len(chunks))

Let's create functions to search through those chunked plays for relevant sections.

In [ ]:
import numpy as np

def cosine_similarity (vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    return dot_product / (norm_vector1 * norm_vector2)

def check_document_relevance(query):
    class Relevance:
        def __init__(self, name, snippet, similarity):
            self.name = name
            self.snippet = snippet
            self.similarity = similarity

    query_embedding = model.encode(query)

    print(f'Relevance for "{query}":')
    print('--------------')

    relevances = []
    for chunk in chunks:
        similarity = cosine_similarity(query_embedding, chunk.embedding)
        relevances.append(Relevance(chunk.doc_name, chunk.text, similarity))
    relevances.sort(key=lambda x: x.similarity, reverse=True)
    for answer in relevances[:5]:
        print(f'{answer.name} ({answer.similarity}): {answer.snippet[:500]}\n')
    print()

In [ ]:
%%time
check_document_relevance("What was Romeo's last name?")
check_document_relevance("Who stabbed Julius Caesar?")

We have a mere 17k vectors but it takes hundreds of milliseconds to search for two query strings. This begins to show the challenge of using cosine similarity directly (which scales linearly) and the need for a more efficient algorithm.

## Example usage

FAISS requires an index. There are many ways of indexing vectors, this notebook will explore some of the simpler ones.

### Flat L2 index

Indexes based on the distance between vectors. This will give the highest quality matches but is slower than other FAISS indeces.

In [ ]:
import faiss
import numpy

# Create the index
index = faiss.IndexFlatL2(len(chunks[0].embedding))

# Flat L2 index doesn't need trained since it is not specific to the data set
print(index.is_trained)

# Add embeddings
index.add(numpy.array([c.embedding for c in chunks]))
print(index.ntotal)

Now, as before, we can search but we'll use `index.search` instead of cosine similarity.

In [ ]:
def check_document_relevance_faiss(query, index):
    class Relevance:
        def __init__(self, name, snippet, similarity):
            self.name = name
            self.snippet = snippet
            self.similarity = similarity

    query_embedding = model.encode([query])

    print(f'Relevance for "{query}":')
    print('--------------')

    # Searching returns both the distances of each vector from the query vector
    # and the indeces for those nearest neighbors
    distances, indeces = index.search(query_embedding, 5)
    
    # Note that relevance with FAISS is different from cosine similarity.
    # Now *lower* values are more relevant instead of higher ones.
    relevances = [Relevance(chunks[i].doc_name, chunks[i].text, distances[0][j]) for j, i in enumerate(indeces[0])]

    for answer in relevances[:5]:
        print(f'{answer.name} ({answer.similarity}): {answer.snippet[:500]}\n')
    print()

In [ ]:
%%time
check_document_relevance_faiss("What was Romeo's last name?", index)
check_document_relevance_faiss("Who stabbed Julius Caesar?", index)

Much better. But it can still be improved! While using a FAISS index to search improves performance over just running cosine similarity over every vector, it still scales linearly and will run into performance problems as the set of vectors we're testing against grows. Let's look at other FAISS index options that can help.

## Flat IP

This is very similar to FlatL2 except that it uses a slightly different algorithm for determining distance between vectors. Flat L2 looks at distance between vectors whereas Flat IP is the inner product (dot product) of the two. Flat IP is like cosine similarity except that it's not normalized (so IP cares about vector magnitude whereas cosine similarity does not).

## Flat IVF Index

To avoid the problem of linearly scaling search times with a flat index, we can partition vectors into groups ("cells") based on their position in vector space. Each group is assigned a centroid which is a centrally-located element in the cell. Now, when we search, FAISS can first identify the closest centroid to the query vector using a flat index approach and then search only that partition for results. This is referred to as an [inverted file index](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes#cell-probe-methods-indexivf-indexes) because the cells are inverted lists.

In some scenarios it might also make sense to identify a small number of centroids closest to the query vector, instead, to account for scenarios where the query vector is near a "boundary" of a partition.

Note that this is an *approximate* search (as opposed to flat L2) since we're not checking all possible matches and could in some cases miss relevant ones. It is, however, much faster.

In [ ]:
cell_count = 10

# Index IVF requires another trained index to use for comparison within clusters and a count
# for the number of clusters
indexIVF = faiss.IndexIVFFlat(index, len(chunks[0].embedding), cell_count)

# IVF indeces require training to assign the centroids.
print("IVF trained initially: ", indexIVF.is_trained)
indexIVF.train(numpy.array([c.embedding for c in chunks]))
print("IVF trained after training: ", indexIVF.is_trained)
indexIVF.add(numpy.array([c.embedding for c in chunks]))
print(indexIVF.ntotal)

In [ ]:
%%time
check_document_relevance_faiss("What was Romeo's last name?", indexIVF)
check_document_relevance_faiss("Who stabbed Julius Caesar?", indexIVF)

The IVF index's nprobe value determines how many centroids to search. A higher nprobe value will search more centroids and return more accurate results but will be slower. A lower nprobe value will be faster but may miss relevant results.

In [ ]:
indexIVF.nprobe = 4

In [ ]:
%%time
check_document_relevance_faiss("What was Romeo's last name?", indexIVF)
check_document_relevance_faiss("Who stabbed Julius Caesar?", indexIVF)

## Product quantization

Another tweak on IVF index is IVF with product quantization. This is a more advanced technique that partitions vectors into sub-vectors, each of which is mapped to a centroid for that portion of the vector. These centroid IDs are then stored in the index. This allows for more efficient storage and search.

In [ ]:
# Determine the number of segments to split the vectors into
# The number must evenly divide the vector length
m = 8

# Bits in the centroid
bits = 8

cell_count = 10

indexIVFPQ = faiss.IndexIVFPQ(index, len(chunks[0].embedding), cell_count, m, bits)
indexIVFPQ.train(numpy.array([c.embedding for c in chunks]))
print(indexIVFPQ.is_trained)
indexIVFPQ.add(numpy.array([c.embedding for c in chunks]))
print(indexIVFPQ.ntotal)

In [ ]:
%%time
check_document_relevance_faiss("What was Romeo's last name?", indexIVFPQ)
check_document_relevance_faiss("Who stabbed Julius Caesar?", indexIVFPQ)

In [ ]:
# Performance comparer
def compare_performance(test_index, query_embedding, search_count):
    distances, indeces = test_index.search(query_embedding, search_count)
    print(indeces)

In [ ]:
test_embeddings = model.encode(["What was Romeo's last name?", "Who stabbed Julius Caesar?", "Where is Hamlet set?", "Who is Oberon?"])

In [ ]:
%%time
compare_performance(index, test_embeddings, 5)

In [ ]:
%%time
compare_performance(indexIVF, test_embeddings, 5)

In [ ]:
%%time
compare_performance(indexIVFPQ, test_embeddings, 5)

Times are all quite close and quite small, but it is clear that the IVF and IVFPQ indeces are faster than the flat L2 index. Note that the results do vary with the flat L2 index being the most accurate and the others being similar but not quite as good.

## Locality Sensitive Hashing (LSH)

Works by hashing vectors into buckets. Unlike other common hashing scenarios, LSH does not try to minimize collisions. Instead, it hashes in such a way that similar vectors are *likely* to collide. Consequently, similar vectors are more likely to be hashed into the same bucket. This is a very fast way to search for similar vectors but is approximate and can miss relevant results.

In [ ]:
d = len(chunks[0].embedding)

# Number of bits to use for the hash
# Higher numbers will have higher hash granularity (and, therefore, both higher accuracy and slower speed)
# Very high nbits values can even make this slower than Flat L2/Flat IP
nbits = int(d / 2)

indexLSH = faiss.IndexLSH(d, nbits)
indexLSH.add(numpy.array([c.embedding for c in chunks]))

print(indexLSH.is_trained)
print(indexLSH.ntotal)

In [ ]:
%%time
compare_performance(indexLSH, test_embeddings, 5)

## Hierarchical Navigable Small World (HNSW)

Navigable small world is a (potentially large) graph with a small number of hops between any two vertices.

Hierarchical NSW is an NSW split across multiple layers.

In [ ]:
# Higher parameters are more accurate, lower are faster
connection_count = 16
search_depth = 16
construction_search_depth = 64 # Slows index creation but not runtime

indexHNSW = faiss.IndexHNSWFlat(d, connection_count)

indexHNSW.hnsw.efSearch = search_depth
indexHNSW.hnsw.efConstruction = construction_search_depth

print(indexHNSW.is_trained)

indexHNSW.add(numpy.array([c.embedding for c in chunks]))

print (indexHNSW.ntotal)

In [ ]:
%%time
compare_performance(indexHNSW, test_embeddings, 5)

## Performance comparison

Let's compare performance of some of these algorithms

In [ ]:
import time

def measure_time(name, iterations, func, *args, **kwargs):
    start_time = time.time()
    for i in range(iterations):
        func(*args, **kwargs)
    end_time = time.time()
    print(f"{name}: {end_time - start_time} seconds")
    return end_time - start_time

In [ ]:
algorithms = {
    'Flat L2': index,
    'Flat IVF': indexIVF,
    'IVF PQ': indexIVFPQ,
    'LSH': indexLSH,
    'HNSW': indexHNSW
}

timings = [(name, measure_time(name, 1000, lambda: test_index.search(test_embeddings, 5))) for name, test_index in algorithms.items()]

import matplotlib.pyplot as plt

keys, values = zip(*timings)
plt.bar(keys, values)
plt.xlabel('Algorithm')
plt.ylabel('Timing (x1000)')
plt.title('Algorithm Timing')
plt.show()